Реализуйте упрощенную версию byte pair encoding (без предварительного разбивания на слова, которое есть в посте о BPE на медиуме –  
https://towardsdatascience.com/byte-pair-encoding-the-dark-horse-of-modern-nlp-eb36c7df4f10).  

Алгоритм должен работать так:  
строки с текстом разбиваются на отдельные символы и далее в цикле из N итераций:  
    а) считаются статистики встречаемости по парам символов и  
    б) топ-K частотных пар склеиваются в один символ  

Попробуйте так токенизировать текст с разными параметрами N и K.  
Проанализируйте словарь уникальных слов для нескольких наборов параметров - сколько уникальных слов получилось, какой токен самый длинный?  

Чтобы получить 1 бонусный балл - зафиксируйте получившийся словарь и токенизируйте с помощью него текст, который ранее не встречался в корпусе (возьмите рандомную новость из яндекс новостей например).  
Проанализируйте насколько хорошо токенизировался текст.

# Задание 1

In [1]:
with open('lenta.txt', 'r', encoding='utf-8') as f:
    text = f.read()
text = text[:50000].lower()

### считаются статистики встречаемости по парам символов

In [2]:
def stats_collect(text):
    freqs = {}
    for i in range(len(text)-1):
        next_symbol = text[i+1]
        pair = text[i] + next_symbol
        if pair in freqs:
            freqs[pair] += 1
        else:
            freqs[pair] = 1
    return freqs

### топ-K частотных пар склеиваются в один символ 

In [3]:
def merger(text, dictionary, k):
    sort_dict = {}
    keys = list(dictionary.keys())
    keys = sorted(keys, key=lambda x: dictionary[x], reverse=True)
    keys = keys[:k]
    for key in keys:
        sort_dict[key] = dictionary[key]
    result = ()
    process = True
    for ind, sym in enumerate(text):
        if process:
            if ind < len(text)-1:
                next_symbol = text[ind+1]
            else:
                result += tuple(sym, )
                break
            pair = sym + next_symbol
            if pair in sort_dict: 
                result += (pair, )
                process = False
            else:
                result += (sym, )
        else:
            process = True
    return result

# вычисляем bpe

In [4]:
def bpe(text, n, k):
    for i in range(n):
        frequencies = stats_collect(text)
        text = merger(text, frequencies, k)
    return text

n = [3, 5, 10, 20, 30]
k = [5, 12, 25, 50, 60]



for i,j in zip(n, k):
    result = bpe(text, i, j)
    print('_______________________________')
    print('n = ' + str(i), ',k = ' + str(j))
    print('n of tokens: ' + str(len(result)))
    print('/'.join(result[:100]))
    print('Longest token: ' + str(sorted(result, key=lambda x: len(x), reverse=True)[0]))

_______________________________
n = 3 ,k = 5
n of tokens: 42881
б/о/и /у/ с/о/п/о/ц/к/и/на/ /и /д/р/у/с/к/ен/и/к/ /з/а/к/о/н/ч/и/л/и/с/ь/ о/т/ст/у/п/л/ен/и/е/м/ /г/е/р/м/а/н/ц/е/в/./ /н/е/п/р/и/я/т/е/л/ь/,/ п/р/и/б/л/и/з/и/в/ш/и/с/ь/ с/ с/е/в/е/ра/ /к/ о/с/ов/ц/у/ /на/ч/а/л/ /а
Longest token: и 
_______________________________
n = 5 ,k = 12
n of tokens: 35121
б/о/и /у/ с/о/п/о/ц/к/ин/а /и /д/р/у/ск/ени/к/ /за/ко/н/ч/и/ли/с/ь/ о/т/ст/у/п/л/ени/е/м /г/ер/м/ан/ц/е/в/./ н/е/п/ри/я/т/ел/ь/, /п/ри/б/ли/з/и/в/ш/и/с/ь/ с/ с/е/в/ер/а /к/ о/с/ов/ц/у/ н/а/ч/ал/ /ар/т/ил/л/ер/и/й/ск/у/ю/ б/ор/ь/б/у/ с/ /к
Longest token: ени
_______________________________
n = 10 ,k = 25
n of tokens: 26758
бо/и /у/ со/по/ц/ки/на /и /д/ру/ск/ени/к /за/кон/ч/ил/ис/ь/ от/сту/п/лени/е/м /г/ер/м/ан/ц/е/в/./ н/е/при/я/тель/, /при/б/ли/зи/вши/с/ь/ с/ се/в/ера/ /к/ о/сов/ц/у/ на/ч/ал/ /ар/т/ил/л/ер/и/й/ск/у/ю/ б/ор/ь/бу/ с/ /к/ре/по/ст/ь/ю/./ в /ар/т/ил/л/ер/и/й/ско/м /бо/ю/ при/ни/ма/ют
Longest token: ности 
_____________

По мере увеличения параметров мы видим, как буквы складываются в "морфемы", "морфемы" в "фрагменты слов" и "слова", "слова" в "словосочетания".
Естественно, что это все происходит во многом случайно (в зависимости от обучающего корпуса), потому что мы никак не оптимизируем процесс отбора слов.

### Задание 2


Обучите токенизатор из tokenizers на текстовом корпусе. 
Рассчитайте статистики для idf по корпусу, используя обученный словарь (разбейте корпус на "документы" по новым 
строкам, каждый "документ" токенизируйте, для каждого слова посчитайте, в скольких документах оно встречается и рассчитайте 
idf разделив общее количество документов на это число, возьмите логарифм от полученного числа). 
Векторизуйте текст (в мешок слов) аналогично TfidfVectorizer, используя токенизатор и idf статистики 
(инициализируйте*** пустую матрицу размером (N документов, K слов в словаре) и в цикле по всем документам постепенно 
заполните ее - токенизируйте документ, рассчитайте TF каждого слова (количество вхождений в документе поделить на общее 
количество слов в документе), умножьте TF на IDF и, используя индексы слов в словаре, запишите получившееся значение в матрицу)

Формулу для TFIDF можете уточнить тут - https://ru.wikipedia.org/wiki/TF-IDF

***Чтобы инициализировать разреженную матрицу используйте scipy:
from scipy.sparse import lil_matrix
X = lil_matrix(N, K)
Обучите классификатор на полученных векторах и оцените на кросс-валидации. 


In [5]:
import re
import numpy as np
import pandas as pd
from scipy.sparse import lil_matrix
from collections import Counter, defaultdict
from tokenizers import CharBPETokenizer, Tokenizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

In [6]:
df = pd.read_csv('dataset_ok.csv')
data = df['text'].tolist()
df['text'].to_csv('data/data.txt', index=False)
subtoken = CharBPETokenizer()
subtoken.train('data/data.txt', vocab_size=3000)
subtoken.save('subtoken')
subtoken = Tokenizer.from_file('subtoken')

N = len(data)
K = len(subtoken.get_vocab())

X = lil_matrix((N, K))
X_idf = lil_matrix((N, K))

for i, text in enumerate(data):
    token_ids = subtoken.encode(text).ids
    for t in token_ids:
        X[i, t] += 1
        if X_idf[i, t] == 0:
            X_idf[i, t] = 1

idf = pd.Series(X_idf.sum(axis=0).tolist()[0])
idf = idf.apply(lambda x: np.log((1 + idf.shape[0]) / (1 + x)) + 1)

X = X.multiply(lil_matrix(idf.tolist()))
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.2, stratify=df['label'])

clf = SGDClassifier(loss="log", max_iter=30)
cross_val_score(clf, X_train, y_train, scoring="f1_micro")

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


array([0.92620247, 0.92403195, 0.92663657, 0.92342421, 0.92871407])